In [2]:
from IPython.display import display

import time
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 300)

import pickle # save file as binary, load & dump much faster than csv

def load_data(filename):
    with open(filename, 'rb') as fin:
        return pickle.load(fin)
    
def save_data(obj, filename):
    with open(filename, 'wb+') as fout:
        pickle.dump(obj, fout)

import os
root_path = 'C:/Users/yuanl4/Documents/MelbDatathon2017/'

In [3]:
def read_patients(i):
    df = pd.read_csv(root_path + 'Final/patient_%d.csv' % i, 
                     parse_dates = ['Dispense_Week'],
                     usecols=['Patient_ID', 'Store_ID', 'Drug_ID', 'Dispense_Week'])
    df.columns = ['PID', 'SID', 'DID', 'DispWeek']
    return df

def convert_to_matrix(df):
    assert(set(df.dtypes.values) == {np.dtype('int64'), np.dtype('<M8[ns]')}) # data dtype is either date or int64

    matrix = np.zeros(df.shape, dtype=np.int32)

    year2000 = np.datetime64('2000-01-01')
    for i, (col_dtype, col_name) in enumerate(zip(df.dtypes.values, df.columns)):
        if col_dtype == np.dtype('int64'):
            matrix[:, i] = df[col_name].values
        else:
            matrix[:, i] = (df[col_name].values - year2000).astype('timedelta64[D]')
    return matrix, df.index.values

def read_data(file_range):
    transactions = []
    for i in file_range:
        print(i, end=' ')
        transactions.append(convert_to_matrix(read_patients(i))[0])
    print('finish loading')

    transactions = np.vstack(transactions)

    transactions[:, 0] -= transactions[:, 0].min() # now patient ID starts at 0
    return transactions

In [4]:
%%time
transactions = read_data(range(1, 51)) # 1~50

num_patients = transactions[-1, 0] + 1
print('patients in the training set:', num_patients)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 finish loading
patients in the training set: 558352
Wall time: 58.3 s


In [5]:
# save_data({'data':transactions, 
#            'cols':['PID', 'SID', 'DID', 'DispWeek']},
#           root_path + 'Summary/all_transactions.pkl')

In [6]:
patients = np.asanyarray(np.split(transactions, np.bincount(transactions[:, 0]).cumsum()[:-1]))

In [7]:
def group_count_range_gap_min_max_median(groups, values):
    unique_groups, count_groups = np.unique(groups, return_counts=True)
    output = np.zeros((unique_groups.shape[0], 6), dtype=np.int16)
    # groups, count, range, gap_min, gap_max, gap_median
    # 1,      1,     1,     1,       1,       1
    output[:, 0] = unique_groups # groups
    output[:, 1] = count_groups  # counts
    for i, g in enumerate(unique_groups):
        values_selected = values[groups==g]
        output[i, 2] = values_selected.max() - values_selected.min() # range
        gaps = np.ediff1d(values_selected)       # gaps
        if gaps.shape[0] != 0:
            output[i, 3] = gaps.min()            # gap min
            output[i, 4] = gaps.max()            # gap max
            output[i, 5] = np.median(gaps)       # gap median
    return output

In [8]:
%%time
summaries = []
print('Number of patients', len(patients))
for i, patient in enumerate(patients):
    if i%10000==0:
        print(i, end=' ')
    summaries.append(group_count_range_gap_min_max_median(patient[:, 2], patient[:, 3])) # DID and DispWeek
print('finished.')

Number of patients 558352
0 10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000 210000 220000 230000 240000 250000 260000 270000 280000 290000 300000 310000 320000 330000 340000 350000 360000 370000 380000 390000 400000 410000 420000 430000 440000 450000 460000 470000 480000 490000 500000 510000 520000 530000 540000 550000 finished.
Wall time: 3min 26s


In [9]:
def merge_summaries(summaries):
    patient_count = np.asarray(list(map(len, summaries)))
    PIDs = np.bincount(patient_count.cumsum()).cumsum()[:-1]
    return np.hstack([PIDs[:, None], np.vstack(summaries)])
patient_drug_summary = merge_summaries(summaries).astype(np.int32)

In [11]:
%%time
df = pd.DataFrame(patient_drug_summary, columns=['PID', 'DID', 'Cnt', 'Dur', 'G.Min', 'G.Max', 'G.Med'])
df.to_csv(root_path+'Summary/patient_drug_summary.csv', index=False)

Wall time: 30.4 s


In [12]:
def value_range_gap_min_max_median(i, values, output):
    if values.shape[0] == 0:
        return output
    # count, range, gap_min, gap_max, gap_median
    # 1,     1,     1,       1,       1
    output[i, 1] = count = values.shape[0]     # counts
    output[i, 2] = values.max() - values.min() # range
    if count > 1:
        gaps = np.ediff1d(values)              # gaps
        output[i, 3] = gaps.min()              # gap min
        output[i, 4] = gaps.max()              # gap max
        output[i, 5] = np.median(gaps)         # gap median
    return output

In [13]:
patient_summary = np.zeros((num_patients, 6), dtype=np.int32) # drug_count, trans_count, range, gap_min, gap_max, gap_median
patient_summary[:, 0] = list(map(len, summaries)) # distinct drugs, drug_count

In [14]:
%%time
print('Number of patients', len(patients))
for i, patient in enumerate(patients):
    if i%10000==0:
        print(i, end=' ')
    value_range_gap_min_max_median(i, patient[:, 3], patient_summary) # patient[:, 3] DispWeek
print('finished.')

Number of patients 558352
0 10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000 210000 220000 230000 240000 250000 260000 270000 280000 290000 300000 310000 320000 330000 340000 350000 360000 370000 380000 390000 400000 410000 420000 430000 440000 450000 460000 470000 480000 490000 500000 510000 520000 530000 540000 550000 finished.
Wall time: 17 s


In [15]:
patient_summary = pd.DataFrame(patient_summary.astype(float),
                               columns=['D.Cnt', 'T.Cnt', 'Dur', 'G.Min', 'G.Max', 'G.Med']) # convert to float to use nan

patient_summary.loc[patient_summary['D.Cnt']==0, ['Dur', 'G.Min', 'G.Max', 'G.Med']] = np.nan
patient_summary.loc[patient_summary['Dur']==0, ['G.Min', 'G.Max', 'G.Med']] = np.nan

In [16]:
patient_summary.to_csv(root_path + 'Summary/patient_summary.csv', index_label='PID')

In [21]:
def summary_for_one_patient(data):
    display(pd.DataFrame(data, columns=['PID', 'SID', 'DID', 'DispWeek']))
    
    output = group_value_range_gap_min_max_median(data[:, 2], data[:, 3])
    return pd.DataFrame(output, columns=['DID', 'Cnt', 'Dur', 'G.Min', 'G.Max', 'G.Med'])

summary_for_one_patient(patients[6])

,PID,SID,DID,DispWeek
0,6,1427,3423,4236
1,6,1427,3423,4271
2,6,257,2440,4691
3,6,2338,12090,5909


,DID,Cnt,Dur,G.Min,G.Max,G.Med
0,2440,1,0,0,0,0
1,3423,2,35,35,35,35
2,12090,1,0,0,0,0
